In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Installs

In [23]:
!pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0
!pip install datasets==2.16.1 PyPDF2==3.0.1 langchain_experimental python-dotenv==1.0.1 pyyaml==6.0.1 coloredlogs==15.0.1 mdc==1.2.1 pytest==8.1.2
!pip install -r drive/MyDrive/RAG-llamaindex/requirements.txt
!pip install huggingface_hub
!pip install llama-index-llms-huggingface
!pip uninstall -y bitsandbytes accelerate transformers
!pip install bitsandbytes accelerate transformers


  Using cached transformers-4.40.2-py3-none-any.whl (9.0 MB)
  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3
Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)


In [24]:
!pip install gradio
!pip install spaces

ERROR: Operation cancelled by user


#Imports


In [2]:
# For Loading the LLM
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
from transformers import AutoModel

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
import torch

import warnings
warnings.filterwarnings('ignore')

# For Data Ingestion
from typing import Literal, Any, List
import logging
import PyPDF2
import sys
sys.path.append('drive/MyDrive/RAG-llamaindex')
import os
from langchain_experimental.text_splitter import SemanticChunker
from math import ceil

# For Embeddings
from transformers import AutoModel

# For the LLM Model
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# for deployment
import gradio as gr
import spaces
from threading import Thread

# for results
import csv
import pandas as pd

# for retrievers
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#Data Ingestion

In [3]:
reader = SimpleDirectoryReader(input_dir="drive/MyDrive/RAG-llamaindex/Data")
documents = reader.load_data(num_workers=4)

# Filter out the data where the file_name is not "Harry Potter The Complete Collection.pdf"
documents = [doc for doc in documents if doc.metadata["file_name"] == "Harry Potter The Complete Collection.pdf"]

# Creating a function for chunking using the LLama Index SentenceSplitter
def get_chunks(
    documents: list[Any],
    chunk_size: int = 512,
    chunk_overlap: float = 0.0
) -> list[str]:
    overlap = int(chunk_size * chunk_overlap)
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks= text_splitter.get_nodes_from_documents(documents, show_progress=True)

    return chunks

# Creating a function for manual chunking
def get_chunks_manual(documents: list[Any], chunk_size: int = 512, chunk_overlap: float = 0.0) -> List[str]:
    """
    Takes in a `file_path`, retrieves the document, breaks it down into chunks of size
    `chunk_size` and overlap `chunk_overlap`, and returns the chunks.
    """
    chunks = []

    text = ""
    for doc in documents:
        text += doc.text
    # replace every \t with " "
    text = text.replace("\t", " ")

    step_size = ceil(chunk_size * (1 - chunk_overlap))
    for i in range(0, len(text), step_size):
        chunks.append(text[i:i + chunk_size])

    return chunks

#Creating the Embedding Model

In [4]:
class HuggingFaceEmbedding:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_text_embedding(self, text: str) -> torch.Tensor:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze()

    def get_embeddings(self, texts: list[str], batch_size: int = 16) -> list[torch.Tensor]:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            inputs = self.tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.extend(batch_embeddings.cpu())
        return embeddings



#Defining the LLM Model

In [5]:
# Creating the first Retriever: Top-k Embeddings Retriever

def top_k_embed_retriever(query: str, k: int, embed_model, manual_embed_np, texts):
    query_embed = embed_model.get_text_embedding(query)
    query_embed_np = query_embed.cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity with all the other embeddings
    similarities = cosine_similarity(query_embed_np, manual_embed_np).flatten()

    # Find the top k indices
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]

    # Retrieve the top k texts
    top_k_texts = [texts[i] for i in top_k_indices]

    return top_k_texts

In [6]:
# Creating the second Retriever: Top-k TF-IDF Retriever
def top_k_tfidf_retriever(query: str, k: int, tfidf_matrix, texts, vectorizer):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]
    top_k_texts = [texts[i] for i in top_k_indices]
    return top_k_texts


#Creating the Pipeline

In [15]:
# Getting the Hugging Face token
config_data = json.load(open("drive/MyDrive/RAG-llamaindex/config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

# Defining the pre-trained model we will use, which is Llama-3-8B
model_name = "meta-llama/Meta-Llama-3-8B"

# Defining the quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Defing the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name ,
                                          token=HF_TOKEN)
# tokenizer.pad_token = tokenizer.eos_token

# Initialising the LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name ,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
SYS_PROMPT = """You are an assistant for answering questions about the Harry Potter books.
You are given the extracted parts of the books and a question. Provide a short answer.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

def format_prompt(prompt,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  PROMPT = f"Question:{prompt}\nContext:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents[idx]}\n"
  return PROMPT

def generate_response(formatted_prompt):
  formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
  messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
  # tell the model to generate
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  # Ensure pad_token_id is set to eos_token_id if not already set
  if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id



  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [18]:
def rag_pipeline(documents: list[Any], df: pd.DataFrame, chunk_sz: int = 512, chunk_ol: float = 0.0, retriever: str = "embed", k: int = 3, embed_model=None):
    '''
    This function takes a dataframe `df` that contains the QA's, `chunk_sz` and overlap `chunk_ol`,
     a retriver `retiever` between the top_k_embed_retriever and the top_k_tfidf_retriever,
    the parameter `k` for the retrievers and an embeddings model.
    '''

    # Creating the chunks
    print("Chunking")
    chunks = get_chunks_manual(documents, chunk_sz, chunk_ol)

    # Manual Embeddings for TF-IDF
    if retriever == 'embed':
      # Loading the embedding model and embed the chunks
      print("Loading Embedding Model")
      manual_embed = embed_model.get_embeddings(chunks)
      manual_embed_np = [embedding.numpy() for embedding in manual_embed]
    else:
      # Calculating TF-IDF
      print("Calculating TF-IDF")
      vectorizer = TfidfVectorizer()
      tfidf_matrix = vectorizer.fit_transform(chunks)

    results = []
    for index, row in df.iterrows():
      query = row["Question"]

      if retriever == "embed":
        top_k_texts = top_k_embed_retriever(query, k, embed_model, np.stack(manual_embed_np), chunks)
      else:
        top_k_texts = top_k_tfidf_retriever(query, k, tfidf_matrix, chunks, vectorizer)

      retrieved_context = " ".join(top_k_texts)


      formatted_prompt = format_prompt(query,top_k_texts,k)
      llm_answer = generate_response(formatted_prompt)

      result = {
          "Question": query,
          "True_Answer": row['Answer'],
          "LLM_Answer": llm_answer,
          "Retrieved_Context": retrieved_context
      }
      print(result)
      results.append(result)

    return pd.DataFrame(results)

In [23]:
embed_model =  HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
df = pd.read_csv('drive/MyDrive/RAG-llamaindex/harry-potter-data.csv')

result_df = rag_pipeline(documents, df, chunk_sz=512, chunk_ol=0.0, retriever="embed", k=3, embed_model=embed_model)
result_df.to_csv('drive/MyDrive/RAG-llamaindex/retriever_results/ans_512_00_embed_3.csv', index=False)
print(result_df.head())


Chunking
Loading Embedding Model


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'Question': "Who is Harry Potter's best friend?", 'True_Answer': 'Ron Weasley', 'LLM_Answer': 'Questionutzer_<?_<?_<?_<?_<?utzerutzer_<?_<?_<?_<?MOOTH_<?_<?utzer_<?utzer_<?utzer_<?_<?_<?utzerutzer470utzer_<?_<?_<?utzer_<?_<?utzeronse姫_<? Destruction_<?_<?MOOTH Shore_<?_<?utzer_<?_<?utzer_<?_<?_<?_<?_<?_<?utzerutzer_<?utzerutzer_<?_<?utzer_<?utzer_<?emouth_<?utzer_<?_<?_<?utzer_<?utzerutzer_<?_<?utzer_<?姫utzer Shore �utzer_<?_<? Destruction_<?utzer_<?_<? Destruction_<? �姫 Destruction Shore_<?_<?utzer_<?utzerutzer Destructionregononse姫_<?onseutzerutzer_<?_<?_<?utzeronse_<?_<?_<?vak_<?utzeremouthutzer_<?_<?_<?_<?regon_<?_<?_<?utzer Destruction Destruction_<?_<?_<?utzer_<?_<?onse_<?_<?_<?_<?_<?utzer_<?_<?utzerutzerutzer DestructionMOOTHutzer_<?utzer_<?utzerutzerutzer_<?vakutzerutzer_<?utzer_<?_<?_<?_<?utzer snaonse_<?_<?_<?_<?utzer_<?_<?utzer_<?utzer_<?utzerutzer_<?utzerutzer姫_<?utzerutzer_<?_<?utzer姫utzerutzerutzer_<?_<?_<? sna姫_<?_<?_<? sna_<?utzer_<?_<?_<?_<?_<?_<?_<?_<?_<?_<?_<?_<?utz

KeyboardInterrupt: 

In [ ]:
embed_model =  HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
df = pd.read_csv('drive/MyDrive/RAG-llamaindex/harry-potter-data.csv')

result_df = rag_pipeline(documents, df, chunk_sz=512, chunk_ol=0.5, retriever="embed", k=3, embed_model=embed_model)
result_df.to_csv('drive/MyDrive/RAG-llamaindex/retriever_results/ans_512_05_embed_3.csv', index=False)
print(result_df.head())

In [ ]:

df = pd.read_csv('drive/MyDrive/RAG-llamaindex/harry-potter-data.csv')

result_df = rag_pipeline(documents, df, chunk_sz=512, chunk_ol=0.5, retriever="idf", k=3)
result_df.to_csv('drive/MyDrive/RAG-llamaindex/retriever_results/ans_512_05_idf_3.csv', index=False)
print(result_df.head())